In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import cv2
from skimage import feature
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications import imagenet_utils

In [ ]:
model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

In [ ]:
def extract_visual_features(image_path, model):
    img = Image.open(image_path)
    img = img.convert('RGB')
    img = img.resize((224, 224))
    img_array = img_to_array(img)
    img_array = preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)
    visual_features = model.predict(img_array)
    return visual_features

In [ ]:
net = cv2.dnn.readNet("/content/yolov3.weights", "yolov3.cfg")
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

def extract_semantic_features(image_path):
    img = cv2.imread(image_path)
    height, width, channels = img.shape

    # Detect objects in the image using YOLO
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Process YOLO outputs
    detected_classes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected with high confidence
                label = classes[class_id]
                detected_classes.append(label)

    # Extract semantic features based on detected classes
    semantic_features = np.zeros(len(classes))
    for detected_class in detected_classes:
        if detected_class in classes:
            class_index = classes.index(detected_class)
            semantic_features[class_index] += 1

    return semantic_features

In [ ]:
def extract_color_features(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    hist = cv2.calcHist([img], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    hist = cv2.normalize(hist, hist).flatten()
    return hist

In [ ]:
def extract_shape_features(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    edges = cv2.Canny(img, 100, 200)
    moments = cv2.moments(edges)
    hu_moments = cv2.HuMoments(moments).flatten()
    aspect_ratio = img.shape[1] / img.shape[0]
    shape_features = np.hstack([hu_moments, aspect_ratio])
    return shape_features

In [ ]:
def extract_contextual_features(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    num_contours = len(contours)
    return np.array([num_contours])


In [ ]:
image_dir = '/content/sample_data/food_dataset_aug'


In [ ]:
all_features = []

In [ ]:
for i, filename in enumerate(os.listdir(image_dir)):
    if filename.endswith('.jpg'):
        image_path = os.path.join(image_dir, filename)
        # Extract features for each image
        visual_features = extract_visual_features(image_path, model)
        semantic_features = extract_semantic_features(image_path)
        color_features = extract_color_features(image_path)
        shape_features = extract_shape_features(image_path)
        contextual_features = extract_contextual_features(image_path)
        # Combine all features into a single feature vector
        feature_vector = np.concatenate((visual_features.reshape(1, -1, 1), semantic_features.reshape(1, -1, 1), color_features.reshape(1, -1, 1), shape_features.reshape(1, -1, 1), contextual_features.reshape(1, -1, 1)), axis=1)
        # Append the image index and feature vector to the list
        all_features.append((i, feature_vector.flatten()))


1/1 [==============================] - 0s 189ms/step


In [ ]:
#all_features_array = np.array(all_features)

In [ ]:
#np.save('extracted_features_1.npy', all_features_array)

In [ ]:
# Convert the list of features to a pandas DataFrame
df = pd.DataFrame(all_features, columns=['Index', 'Image Features'])


In [ ]:
# Save the DataFrame to a CSV file
df.to_csv('image_features_aug.csv', index=False)